In [1]:
import pandas as pd 
import numpy as np
import json
import random
import pickle  
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [2]:
nltk.download('omw-1.4')
nltk.download("punkt")
nltk.download("wordnet")
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package omw-1.4 to /home/henda/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/henda/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/henda/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
with open('datas/data1.json') as f:
    data = json.load(f)

In [4]:
words = []
classes = []
corpus = []
  
ignore_letters = ["?", "!", ".", ","]
  
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        #tokenize each word
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        #add documents in the corpus
        corpus.append((word_list, intent["tag"]))
        # add to our classes list
        if intent["tag"] not in classes:
            classes.append(intent["tag"])
# lemmaztize and lower each word and remove duplicates            
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
  
words = sorted(set(words))
classes = sorted(set(classes))

In [5]:
len(corpus)

122

In [6]:
len(classes)

43

In [7]:
len(words)

239

In [8]:
df = pd.DataFrame(corpus)
df

,0,1
0,"[Hi, there]",greeting
1,"[How, are, you]",greeting
2,"[Is, anyone, there, ?]",greeting
3,[Hey],greeting
4,[Hola],greeting
...,...,...
117,"[Where, is, Project, head, department, ?]",cabin
118,"[where, is, cafeteria]",cabin
119,"[guide, me, to, the, canteen]",cabin
120,"[How, to, improve, team, members, domain, know...",domain


In [9]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [10]:
dataset = []
template = [0]*len(classes)

for document in corpus:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]

    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(template)
    output_row[classes.index(document[1])] = 1
    dataset.append([bag, output_row])

random.shuffle(dataset)
dataset = np.array(dataset)

/home/henda/anaconda3/envs/final_project/lib/python3.6/site-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [11]:
len(bag)

239

In [12]:
df1 = pd.DataFrame(dataset)
df1

,0,1
0,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
...,...,...
117,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
118,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
119,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
120,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [13]:
train_x = list(dataset[:, 0])
train_y = list(dataset[:, 1])

In [14]:
model = Sequential()
model.add(Dense(256, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

model.save("chatbot_model.h5", hist)

Epoch 1/200
25/25 [==============================] - 1s 2ms/step - loss: 3.7692 - accuracy: 0.0164
Epoch 2/200
25/25 [==============================] - 0s 2ms/step - loss: 3.6410 - accuracy: 0.1148
Epoch 3/200
25/25 [==============================] - 0s 2ms/step - loss: 3.5520 - accuracy: 0.1230
Epoch 4/200
25/25 [==============================] - 0s 2ms/step - loss: 3.4116 - accuracy: 0.1148
Epoch 5/200
25/25 [==============================] - 0s 2ms/step - loss: 3.2574 - accuracy: 0.1557
Epoch 6/200
25/25 [==============================] - 0s 2ms/step - loss: 3.0708 - accuracy: 0.2377
Epoch 7/200
25/25 [==============================] - 0s 2ms/step - loss: 2.9462 - accuracy: 0.2295
Epoch 8/200
25/25 [==============================] - 0s 2ms/step - loss: 2.7278 - accuracy: 0.3033
Epoch 9/200
25/25 [==============================] - 0s 2ms/step - loss: 2.5538 - accuracy: 0.3033
Epoch 10/200
25/25 [==============================] - 0s 2ms/step - loss: 2.2707 - accuracy: 0.4098
Epoch 11/

25/25 [==============================] - 0s 2ms/step - loss: 0.0979 - accuracy: 0.9836
Epoch 84/200
25/25 [==============================] - 0s 2ms/step - loss: 0.0578 - accuracy: 0.9836
Epoch 85/200
25/25 [==============================] - 0s 2ms/step - loss: 0.0497 - accuracy: 0.9918
Epoch 86/200
25/25 [==============================] - 0s 2ms/step - loss: 0.0518 - accuracy: 0.9918
Epoch 87/200
25/25 [==============================] - 0s 2ms/step - loss: 0.0599 - accuracy: 0.9918
Epoch 88/200
25/25 [==============================] - 0s 2ms/step - loss: 0.0661 - accuracy: 0.9836
Epoch 89/200
25/25 [==============================] - 0s 2ms/step - loss: 0.0668 - accuracy: 0.9836
Epoch 90/200
25/25 [==============================] - 0s 2ms/step - loss: 0.0903 - accuracy: 0.9672
Epoch 91/200
25/25 [==============================] - 0s 2ms/step - loss: 0.0562 - accuracy: 0.9918
Epoch 92/200
25/25 [==============================] - 0s 2ms/step - loss: 0.1029 - accuracy: 0.9754
Epoch 93/200


25/25 [==============================] - 0s 2ms/step - loss: 0.0433 - accuracy: 0.9836
Epoch 165/200
25/25 [==============================] - 0s 2ms/step - loss: 0.0418 - accuracy: 0.9836
Epoch 166/200
25/25 [==============================] - 0s 2ms/step - loss: 0.0230 - accuracy: 1.0000
Epoch 167/200
25/25 [==============================] - 0s 2ms/step - loss: 0.0740 - accuracy: 0.9754
Epoch 168/200
25/25 [==============================] - 0s 2ms/step - loss: 0.0232 - accuracy: 1.0000
Epoch 169/200
25/25 [==============================] - 0s 2ms/step - loss: 0.0340 - accuracy: 0.9918
Epoch 170/200
25/25 [==============================] - 0s 2ms/step - loss: 0.0465 - accuracy: 0.9836
Epoch 171/200
25/25 [==============================] - 0s 2ms/step - loss: 0.0341 - accuracy: 0.9836
Epoch 172/200
25/25 [==============================] - 0s 2ms/step - loss: 0.0573 - accuracy: 0.9754
Epoch 173/200
25/25 [==============================] - 0s 2ms/step - loss: 0.0536 - accuracy: 0.9918
Epoc

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               61440     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 43)                5547      
Total params: 99,883
Trainable params: 99,883
Non-trainable params: 0
_________________________________________________________________


In [16]:
score = model.evaluate(np.array(train_x), np.array(train_y), verbose=0)
score

[0.015589095652103424, 0.9918032884597778]

In [17]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [18]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [19]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [20]:
def get_response(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [ ]:
for i in range(5):
    message = input("")
    ints = predict_class(message, model)
    res = get_response(ints, data)
    print(res)